![](https://img.kaikeba.com/web/kkb_index/img_index_logo.png)

# 人工智能基础课机器学习 第二节课作业 股票数据清洗

同学们好，本次作业的主要内容为使用pandas进行数据预处理。希望这两天你们已经从Python的课程内容中回过神来了。
没有数据的分析是无源之水，能够熟练处理数据是成为合格的数据分析师的基本要求，希望大家在今后多多实战，成为数据大师。

本次作业将使用公开标普500的股票数据。
https://www.kaggle.com/dgawlik/nyse#prices-split-adjusted.csv

作业的形式以问答为主，因为不涉及过长的代码，核心是数据的操作，所以这里没有太多的代码模板可供大家参考。
希望大家能从搜索引擎（google/bing）问答社区（stackoverflow）或者[官方文档](https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html)或者我提供的jupyter notebooks 中找到回答问题需要的方法。
请时刻牢记数据的增删改查四种原子操作，思考我们的问题可以被如何分解。

那么首先，我们要import pandas

In [1]:
import pandas as pd

# 1. 从fundemantals.csv开始！

fundemantals.csv 是这些股票的年报数据

请用数据回答以下问题：

1. S&P500股票在2015年`net income`的均值是多少？最大值比最小值多多少？（每问10分，共计20分）
2. S&P500股票在2016年的固定资产（fixed assets）占总资产(total assets)比例的均值是多少？固定资产占总资产比例最小的股票是的代码（ticker symbol）是什么？（每问10分，共计20分）


In [1]:

df = pd.read_csv('fundamentals.csv')
df_2015 = df[(df['Period Ending']>='2015-01-01')&(df['Period Ending']<='2015-12-31')]#筛选出15年的数据
ans1 = df_2015['Net Income'].mean()
ans2 = df_2015['Net Income'].max() - df_2015['Net Income'].min()
print(ans1,'\n',ans2)
df_2016 = df[(df['Period Ending']>='2016-01-01')&(df['Period Ending']<='2016-12-31')]#筛选出16年的数据
ans3 = (df_2016['Fixed Assets']/df_2016['Total Assets']).mean()
row_index = (df_2016['Fixed Assets']/df_2016['Total Assets']).idxmin()#获取比列最小的行索引
ans4 = df.iloc[row_index]['Ticker Symbol']
print(ans3,'\n',ans4)

1560252997.752809 
 76922000000.0
0.24428208876549706 
 LEN


1.S&P500股票在2015年net income的均值是1560252997.752809,最大值比小值多76922000000
2.S&P500股票在2016年的固定资产（fixed assets）占总资产(total assets)比例的均值是0.24428208876549706 ,
固定资产占总资产比例最小的股票是的代码（ticker symbol）是LEN

# 2. 加入securities.csv~

securities.csv包含了这些股票的基本信息

1. 请列举出各个sector中的加入时间最早的股票名称（10分）
2. 请列举出每一个州中加入时间最晚的股票名称（10分）


In [9]:
def first_date(df,column='Date first added'):#返回加入时间最早的股票信息
    return df.sort_values(by=column)[0:1]
def last_date(df,column='Date first added'):#返回加入时间最晚的股票信息
    return df.sort_values(by=column)[-1:]
df_all = pd.read_csv('securities.csv')
for i,idata in enumerate(df_all['Address of Headquarters'][0:100]):#把Address of Headquarters这一列分裂成两列city 和 state
    df_all.loc[i,'city'] = idata.split(',')[0]
    df_all.loc[i,'state'] = idata.split(',')[1]
df1 = df_all.dropna(axis = 0, how= 'any',subset =['Date first added'])#去掉Date first added该列为空的行
df2 = df1.groupby(['GICS Sector']).apply(first_date)[['Security','Date first added']]
print('1.各个sector中的加入时间最早的股票名称如下：','\n',df2,'\n')
df3 = df1.groupby(['state']).apply(last_date)[['Security','Date first added']]
print('2.每一个州中加入时间最晚的股票名称如下：','\n',df3)


1.各个sector中的加入时间最早的股票名称如下： 
                                                     Security Date first added
GICS Sector                                                                  
Consumer Discretionary      296             McDonald's Corp.       1970-06-30
Consumer Staples            433           The Clorox Company       1969-03-31
Energy                      399            Schlumberger Ltd.       1965-03-31
Financials                  255         JPMorgan Chase & Co.       1975-06-30
Health Care                 1            Abbott Laboratories       1964-03-31
Industrials                 230         Honeywell Int'l Inc.       1964-03-31
Information Technology      233                      HP Inc.       1974-12-31
Materials                   404             Sherwin-Williams       1964-06-30
Real Estate                 56   AvalonBay Communities, Inc.       2007-01-10
Telecommunications Services 51                      AT&T Inc       1983-11-30
Utilities                   152    

# 3. merge!

现在你需要同时处理来自两个表中的信息了

1. 请思考，合并两个表的信息的时候，我们应该用什么样的准则对其它们（10分）
2. 请列举每个sector在2013-2016年累计Research&Development的总投入（10分）
3. 请列举出每个sector中，在2013-2016年累计Research&development投入最大的3家公司的名称以及投入的数值（20分）

1.请思考，合并两个表的信息的时候，我们应该用什么样的准则对其它们：
两个表有共同列Ticker symbol,但fundamentals中的Ticker symbol列股票代码是重复的，有更多行数，所以我认为应该用 how = left规则来合并。

In [32]:
df_f = pd.read_csv('fundamentals.csv')
df_s = pd.read_csv('securities.csv')
df_s.columns = ['Ticker Symbol','Security', 'SEC Filings', 'GICS Sector',
       'GICS Sub Industry', 'Address of Headquarters', 'Date First Added',
       'CIK']#把列名都修改成单词首字母大写，这样两个表才有共同的列Ticker Symbol，也为了之后都按照首字母大写的列名方式调用，这样调用方便
#print(df_f.columns,'\n',df_s.columns)
df = pd.merge(df_f,df_s,how = 'left')
df1 = df[(df['Period Ending']>='2013-01-01') & (df['Period Ending']<='2016-12-31')]#筛选出13-16年的数据
df2 = df1['Research and Development'].groupby(df1['GICS Sector']).sum()
print('2.每个sector在2013-2016年累计Research&Development的总投入:','\n',df2,'\n')
def largest3(df,column='Research and Development'):#返回R&D最大3家公司的信息
    return df.sort_values(by=column)[-3:]
df3 = df1[df1['Research and Development']>0]#去除掉R&D为0的公司
df4 = df3.groupby(['GICS Sector']).apply(largest3)[['Ticker Symbol','Security','Research and Development']]
print('3.每个sector中，在2013-2016年累计Research&development投入最大的3家公司的名称以及投入的数值:','\n',df4,'\n')

2.每个sector在2013-2016年累计Research&Development的总投入: 
 GICS Sector
Consumer Discretionary         1.860178e+09
Consumer Staples               7.150872e+09
Energy                         2.076513e+09
Financials                     2.401300e+08
Health Care                    1.688513e+11
Industrials                    5.533400e+10
Information Technology         3.158830e+11
Materials                      2.061621e+10
Real Estate                    1.162100e+08
Telecommunications Services    0.000000e+00
Utilities                      0.000000e+00
Name: Research and Development, dtype: float64 

3.每个sector中，在2013-2016年累计Research&development投入最大的3家公司的名称以及投入的数值: 
                             Ticker Symbol              Security  \
GICS Sector                                                       
Consumer Discretionary 717           GRMN           Garmin Ltd.   
                       718           GRMN           Garmin Ltd.   
                       719           GRMN           Garmin Ltd.   
C

# 4. 现在让我们来看看更加复杂的数据

请导入price.csv，然后结合你的聪明才智回答以下问题（附加题，40分）

假设你是某基金公司的老板，现在对于每只股票，你都专门安排了一位负责它的交易员。公司规定每一位交易员手中的资金要么全部买入要么全部卖出（空仓，转化为现金）。假设2016年每一位交易员手中都有10000美元，假设他们都能够看到2016年全年的数据，假设他们都能抓住每一次机会，那么请问2016年底时，赚钱最多的股票是哪一只，赚了多少钱？

In [16]:
#计算思路如下：首先计算出每只股票每天高低价收益率，然后把每只股票每天的收益率汇总求和，最大的即为赚钱最多的股票，
#这个方法是按照单利计算，也就是说每笔投资只投入本金10000美元，不投入赚取的利润。
df = pd.read_csv('prices-split-adjusted.csv')
df['hl_rate'] = round((df['high']/df['low'] -1),3)#增加一列hl_rate,最高最低价的收益率
s1 = df.groupby(['symbol'])['hl_rate'].sum().sort_values(ascending = False)
profit = round(s1.values[0] * 10000,1)
print(s1,'\n')
print('赚钱最多的股票是{0}，赚了{1}美元'.format(s1.index[0],profit))


symbol
FSLR    76.104
CHK     75.773
AAL     72.285
MU      68.066
NFLX    67.205
         ...  
HPE      9.602
CSRA     8.673
KHC      7.355
WLTW     5.134
FTV      2.716
Name: hl_rate, Length: 501, dtype: float64 

赚钱最多的股票是FSLR，赚了761040.0美元
